## 算法说明
1） 在原始特殊的基础上增加，每个grid以及每个grid每小时的最大值、最小值、标准差、偏度、峰度等统计值  
2） 使用lightgbm、RandomForestRegressor、XGBRegressor、Ridge、LinearRegression、BaggingRegressor、GradientBoostingRegressor 7个模型，对所有数据分别训练、预测，将预测结果以及这些结果的差值作为新增的特征加到数据集中；  
3） 再用lgb、xgb做训练、预测；  
4） 融合lgb、xgb的预测结果做最后结果

In [128]:
import pandas as pd
import numpy as np
import warnings
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
warnings.filterwarnings('ignore')
RANDOM_SEED = 42
import lightgbm as lgb
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
random.seed(RANDOM_SEED)
import gc

In [3]:
all_data = pd.read_csv('../huan/pr_qf_feature_v3/PR_QF_BOT_all_data_feature_sum_a.csv')

In [4]:
le = LabelEncoder()
all_data['weather'] = le.fit_transform(all_data['weather'])

In [5]:
all_data.loc[all_data.holiday_counter > 0, 'holiday_counter'] = all_data.loc[all_data.holiday_counter > 0, 'holiday_counter'] \
                                                                            / all_data.loc[all_data.holiday_counter > 0, 'holiday_days']

In [6]:
all_data.loc[all_data.school_holiday_counter > 0, 'school_holiday_counter'] = all_data.loc[all_data.school_holiday_counter > 0, 'school_holiday_counter'] \
                                                                                            / all_data.loc[all_data.school_holiday_counter > 0, 'school_holiday_days']

In [7]:
scaler = MinMaxScaler()
feature_standard_scaler_col = ['temperature','humidity',\
                               'wind_speed',\
                               'holiday_days','school_holiday_days','dew_point','pressure']
all_data[feature_standard_scaler_col] = scaler.fit_transform(all_data[feature_standard_scaler_col])

In [8]:
train_data = all_data[all_data.ymd < '2017-03-13']
test_data = all_data[all_data.ymd >= '2017-03-13']

In [9]:
train_all_data = train_data.copy()

In [10]:
def add_max(grid,hour):
    grid = train_all_data[train_all_data["grid_id"]==grid]
    grid = grid[grid["hour"]==hour]
    return grid["y"].max()
def add_min(grid,hour):
    grid = train_all_data[train_all_data["grid_id"]==grid]
    grid = grid[grid["hour"]==hour]
    return grid["y"].min()
def add_std(grid,hour):
    grid = train_all_data[train_all_data["grid_id"]==grid]
    grid = grid[grid["hour"]==hour]
    return grid["y"].std()
def add_mean(grid,hour):
    grid = train_all_data[train_all_data["grid_id"]==grid]
    grid = grid[grid["hour"]==hour]
    return grid["y"].mean()
def add_kurt(grid,hour):
    grid = train_all_data[train_all_data["grid_id"]==grid]
    grid = grid[grid["hour"]==hour]
    return grid["y"].kurt()

def add_skew(grid,hour):
    grid = train_all_data[train_all_data["grid_id"]==grid]
    grid = grid[grid["hour"]==hour]
    return grid["y"].skew()

In [11]:
def add_max_2(grid):
    grid = train_all_data[train_all_data["grid_id"]==grid]

    return grid["y"].max()
def add_min_2(grid):
    grid = train_all_data[train_all_data["grid_id"]==grid]
    return grid["y"].min()
def add_std_2(grid):
    grid = train_all_data[train_all_data["grid_id"]==grid]
    return grid["y"].std()
def add_mean_2(grid):
    grid = train_all_data[train_all_data["grid_id"]==grid]
    return grid["y"].mean()

def add_kurt_2(grid):
    grid = train_all_data[train_all_data["grid_id"]==grid]
    return grid["y"].kurt()

def add_skew_2(grid):
    grid = train_all_data[train_all_data["grid_id"]==grid]
    return grid["y"].skew()

In [12]:
all_data["add_max"] = all_data.apply(lambda row: add_max(row['grid_id'], row['hour']), axis=1)
all_data["add_min"] = all_data.apply(lambda row: add_min(row['grid_id'], row['hour']), axis=1)

all_data["add_std"] = all_data.apply(lambda row: add_std(row['grid_id'], row['hour']), axis=1)
all_data["add_mean"] = all_data.apply(lambda row: add_mean(row['grid_id'], row['hour']), axis=1)

all_data["add_kurt"] = all_data.apply(lambda row: add_kurt(row['grid_id'], row['hour']), axis=1)
all_data["add_skew"] = all_data.apply(lambda row: add_skew(row['grid_id'], row['hour']), axis=1)

In [13]:
all_data["add_max_2"] = all_data.apply(lambda row: add_max_2(row['grid_id']), axis=1)
all_data["add_min_2"] = all_data.apply(lambda row: add_min_2(row['grid_id']), axis=1)

all_data["add_std_2"] = all_data.apply(lambda row: add_std_2(row['grid_id']), axis=1)
all_data["add_mean_2"] = all_data.apply(lambda row: add_mean_2(row['grid_id']), axis=1)

all_data["add_kurt_2"] = all_data.apply(lambda row: add_kurt_2(row['grid_id']), axis=1)
all_data["add_skew_2"] = all_data.apply(lambda row: add_skew_2(row['grid_id']), axis=1)

In [14]:
train_data = all_data[all_data.ymd < '2017-03-13']
test_data = all_data[all_data.ymd >= '2017-03-13']
train_all_data = train_data.copy()

In [15]:
categorical_feature_col = ['weekday','grid_id','hour','weather','holiday','holiday_bf_af','school_holiday','school_holiday_bf_af']

In [49]:
all_data2 = all_data.copy()

In [50]:
all_data.head()

,grid_id,ymd,weekday,month,day,hour,holiday,holiday_counter,holiday_days,holiday_bf_af,...,add_std,add_mean,add_kurt,add_skew,add_max_2,add_min_2,add_std_2,add_mean_2,add_kurt_2,add_skew_2
0,1,2017-01-02,0,1,2,9,1,1.0,0.428571,0,...,6.515168,19.957143,0.148984,0.692111,40.0,0.0,7.707762,12.714286,-0.589486,0.192291
1,1,2017-01-02,0,1,2,10,1,1.0,0.428571,0,...,6.057779,19.357143,0.460322,0.433468,40.0,0.0,7.707762,12.714286,-0.589486,0.192291
2,1,2017-01-02,0,1,2,11,1,1.0,0.428571,0,...,5.244656,18.171429,0.002011,-0.079883,40.0,0.0,7.707762,12.714286,-0.589486,0.192291
3,1,2017-01-02,0,1,2,12,1,1.0,0.428571,0,...,4.903542,17.314286,-0.408345,-0.063325,40.0,0.0,7.707762,12.714286,-0.589486,0.192291
4,1,2017-01-02,0,1,2,13,1,1.0,0.428571,0,...,5.376604,17.071429,0.028195,0.281969,40.0,0.0,7.707762,12.714286,-0.589486,0.192291


In [51]:
all_data2 = pd.get_dummies(all_data2, columns=categorical_feature_col,drop_first=False)

In [52]:
if 'grid_id_1' in all_data2.columns.tolist():
    print("True")
else:
    print("False")

True


In [185]:
col = all_data2.columns.tolist()

In [186]:
for ii in ["ymd","month","day","y"]:
    col.remove(ii)

In [55]:
minScaler = MinMaxScaler()
all_data2[col] = minScaler.fit_transform(all_data2[col])

In [56]:
train_data = all_data2[all_data.ymd < '2017-03-13']
test_data = all_data2[all_data.ymd >= '2017-03-13']
train_all_data = all_data2.copy()

val_data = train_data[train_data.ymd >= '2017-02-27']
train_data = train_data[train_data.ymd < '2017-02-27']

In [22]:
## 使用lightGBm

In [57]:
train_Y = train_data['y'].values
train_x = train_data[col]

lgb_params = {
    'task': 'train',
    'boosting': 'gbdt',
    'objective': "mse",
    'metric': 'l2',
    'learning_rate': 0.01,
    'verbose': 0,
    'bagging_seed': RANDOM_SEED,
    'num_boost_round': 1500,
 }
dtrain = lgb.Dataset(train_x, train_Y)
model = lgb.train(lgb_params, dtrain)

In [58]:
all_data2["pre_lgb"] = model.predict(all_data2[col])

In [59]:
import xgboost as xgb
xgb_model = xgb.XGBRegressor(n_estimators=1500)
xgb_model.fit(train_x,train_Y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [60]:
all_data2["pre_xgb"] = xgb_model.predict(all_data2[col])

In [61]:
from sklearn.ensemble import RandomForestRegressor

In [62]:
rf = RandomForestRegressor(n_estimators=1000,criterion='mse',random_state=RANDOM_SEED)

In [64]:
rf.fit(train_x,train_Y)
all_data2["pre_rf"] = rf.predict(all_data2[col])

In [65]:
from sklearn.linear_model import Ridge,LinearRegression

In [66]:
lr = LinearRegression()
lr.fit(train_x, train_Y)
all_data2["pre_lr"] = lr.predict(all_data2[col])

In [67]:
rg = Ridge()
rg.fit(train_x, train_Y)
all_data2["pre_rg"] = rg.predict(all_data2[col])

In [68]:
from sklearn.ensemble import BaggingRegressor

In [69]:
bgr = BaggingRegressor(n_estimators=750, random_state=RANDOM_SEED)
bgr.fit(train_x, train_Y)
all_data2["pre_bgr"] = bgr.predict(all_data2[col])

In [70]:
gbdt = GradientBoostingRegressor(n_estimators=750,criterion="mse")
gbdt.fit(train_x, train_Y)
all_data2["pre_gbdt"] = gbdt.predict(all_data2[col])

In [72]:
print("ok")


ok


In [73]:
all_data2.to_csv("./feature_all_data2.csv", index=False)

In [74]:
fc2 = ["pre_gbdt","pre_bgr","pre_rg","pre_lr","pre_rf","pre_xgb","pre_lgb"]

## fc2两两求差值

In [75]:
def get_sub_feature(feature1, feature2):
    if feature1==feature2: 
        return 
    
    sub_feature="{}_sub_{}".format(feature1, feature2)
    print(sub_feature)
    all_data2[sub_feature]=all_data2[feature1]-all_data2[feature2]


In [84]:
for index1, feature1 in enumerate(fc2):
    for index2, feature2 in enumerate(fc2[index1+1:]):
        get_sub_feature(feature1, feature2)

pre_gbdt_sub_pre_bgr
pre_gbdt_sub_pre_rg
pre_gbdt_sub_pre_lr
pre_gbdt_sub_pre_rf
pre_gbdt_sub_pre_xgb
pre_gbdt_sub_pre_lgb
pre_bgr_sub_pre_rg
pre_bgr_sub_pre_lr
pre_bgr_sub_pre_rf
pre_bgr_sub_pre_xgb
pre_bgr_sub_pre_lgb
pre_rg_sub_pre_lr
pre_rg_sub_pre_rf
pre_rg_sub_pre_xgb
pre_rg_sub_pre_lgb
pre_lr_sub_pre_rf
pre_lr_sub_pre_xgb
pre_lr_sub_pre_lgb
pre_rf_sub_pre_xgb
pre_rf_sub_pre_lgb
pre_xgb_sub_pre_lgb


In [86]:
all_data2.to_csv("./feature_all_data2_sub.csv", index=False)

In [189]:
fc = all_data2.columns.tolist()
for ii in ["ymd","month","day","y"]:
    fc.remove(ii)

In [88]:
train_data = all_data2[all_data2.ymd < '2017-03-13']
test_data = all_data2[all_data2.ymd >= '2017-03-13']
train_all_data = train_data.copy()

In [ ]:
# 暂时没用多项式特征
#from sklearn.preprocessing import PolynomialFeatures

In [ ]:
'''
poly = PolynomialFeatures()
poly.fit(train_all_data[fc2])
train_ploy = poly.transform(train_all_data[fc2])
train_ploy = pd.DataFrame(train_ploy, columns=poly.get_feature_names())
X_test_poly = poly.transform(test_data[fc2])
X_test_poly = pd.DataFrame(X_test_poly, columns=poly.get_feature_names())
'''

In [ ]:
#X_test_poly.shape,test_data.shape

In [ ]:
#test_data.reset_index(inplace=True)
#train_data.reset_index(inplace=True)

In [ ]:
#test_data = pd.concat([test_data,X_test_poly],axis=1)

In [ ]:
#train_ploy = pd.concat([train_data,train_ploy],axis=1)

In [ ]:
#fc = train_ploy.columns.tolist()
#for ii in ["ymd","month","day","y"]:
#    fc.remove(ii)

In [ ]:
#train_x = train_ploy[fc]
#train_y = train_ploy["y"]
#test_x = test_data[fc]

In [89]:
train_y = train_all_data['y'].values
train_x = train_all_data[fc]
test_x = test_data[fc]

In [90]:
from sklearn.feature_selection import SelectFromModel

sfm = SelectFromModel(GradientBoostingRegressor(random_state=RANDOM_SEED))
X_train = sfm.fit_transform(train_x, train_y)
X_test = sfm.transform(test_x)

In [95]:
X_train.shape,train_x.shape

((49000, 23), (49000, 137))

In [96]:
X_test.shape,test_x.shape

((9800, 23), (9800, 137))

In [130]:
lgb_model = lgb.LGBMRegressor(boosting_type="gbdt",n_estimators=900)

In [131]:
#lgb_model.fit(X_train,train_y)  #选择特征后的训练数据
lgb_model.fit(train_x,train_y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=900, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

### 线下验证

In [133]:
pre_train = lgb_model.predict(train_x)
print ("modele: LGB RMSE :",np.sqrt(metrics.mean_squared_error(train_y, pre_train)))
print ("modele: LGB RMSE CEIL:",np.sqrt(metrics.mean_squared_error(train_y, np.ceil(pre_train))))
print ("modele: LGB RMSE round:",np.sqrt(metrics.mean_squared_error(train_y, np.round(pre_train))))
print ("modele: LGB RMSE floor:",np.sqrt(metrics.mean_squared_error(train_y, np.floor(pre_train))))

modele: LGB RMSE : 0.4907682905687564
modele: LGB RMSE CEIL: 0.774109099519689
modele: LGB RMSE round: 0.5480205567423952
modele: LGB RMSE floor: 0.7766621412381661


### 查看特征重要性

In [172]:
def analyze_importances(importnce_weight, feature_name):
    pd_fc_w=pd.DataFrame(data=feature_name, columns=['feature'])
    pd_fc_w['weight']=importnce_weight
    
    result = pd_fc_w.sort_values(by=['weight'],ascending=False)
    
    return result.reset_index(drop=True)

In [180]:
lgb_fc_importance=analyze_importances(lgb_model.feature_importances_,fc)
lgb_fc_importance.head()

,feature,weight
0,pre_bgr_sub_pre_rf,1357
1,pre_gbdt_sub_pre_xgb,1105
2,pre_rg_sub_pre_lr,1048
3,pre_gbdt_sub_pre_lgb,900
4,pre_rf,889


### 去除特征重要性为0的特征，重跑模型

In [226]:
fc_zero=lgb_fc_importance[lgb_fc_importance['weight']==0]['feature'].values
print(fc_zero)

fc3=fc.copy()
for ii in fc_zero:
    fc3.remove(ii)

['grid_id_7' 'holiday_0' 'grid_id_11' 'grid_id_1' 'grid_id_14'
 'grid_id_43' 'grid_id_44' 'grid_id_45' 'school_holiday_1'
 'school_holiday_0' 'weather_6' 'holiday_1']


In [192]:
len(fc), len(fc3)

(137, 125)

In [227]:
test_data = all_data2[all_data2.ymd >= '2017-03-13']

train_y2 = train_all_data['y'].values
train_x2 = train_all_data[fc3]
test_x2 = test_data[fc3]

In [203]:
lgb_model2 = lgb.LGBMRegressor(boosting_type="gbdt",n_estimators=750)

#lgb_model.fit(X_train,train_y)  #选择特征后的训练数据
lgb_model2.fit(train_x2,train_y2)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=750, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [204]:
pre_train2 = lgb_model2.predict(train_x2)
print ("modele: LGB RMSE :",np.sqrt(metrics.mean_squared_error(train_y2, pre_train2)))
print ("modele: LGB RMSE CEIL:",np.sqrt(metrics.mean_squared_error(train_y2, np.ceil(pre_train2))))
print ("modele: LGB RMSE round:",np.sqrt(metrics.mean_squared_error(train_y2, np.round(pre_train2))))
print ("modele: LGB RMSE floor:",np.sqrt(metrics.mean_squared_error(train_y2, np.floor(pre_train2))))

modele: LGB RMSE : 0.5226635514837483
modele: LGB RMSE CEIL: 0.7922842202860573
modele: LGB RMSE round: 0.580657614525223
modele: LGB RMSE floor: 0.7935196717081153


### lgb预测

In [222]:
def model_predict(model, test_x, lable="",date=20181001):
    pre = model.predict(test_x)
    test_data = all_data[all_data.ymd >= '2017-03-13'] #重取test_data，为了'grid_id'和'hour'
    
    test_pred_pd = test_data[['grid_id','ymd','hour']].copy()
    test_pred_pd['car_number'] = 0
    
    test_pred_pd['car_number'] = np.ceil(pre)
    test_pred_pd['day'] = pd.to_datetime(test_pred_pd.ymd, format='%Y-%m-%d').dt.strftime('%Y%m%d')
    test_pred_pd = test_pred_pd.sort_values(by=['day','hour','grid_id'])
    
    test_pred_pd[['grid_id','day','hour','car_number']].to_csv('result/{}_result_{}.csv'.format(lable,date),index = False)


In [228]:
model_predict(lgb_model2,test_x2, "lgb", 20181004)

下面的代码不运行，整理到上面函数中

#pre = lgb_model.predict(X_test) #对应特殊选择后的测试数据
pre = lgb_model.predict(test_x)

test_data = all_data[all_data.ymd >= '2017-03-13'] #重取test_data，为了'grid_id'和'hour'

test_pred_pd = test_data[['grid_id','ymd','hour']].copy()
test_pred_pd['car_number'] = 0

test_pred_pd['car_number'] = np.ceil(pre)

test_pred_pd['day'] = pd.to_datetime(test_pred_pd.ymd, format='%Y-%m-%d').dt.strftime('%Y%m%d')

test_pred_pd = test_pred_pd.sort_values(by=['day','hour','grid_id'])

test_pred_pd[['grid_id','day','hour','car_number']].to_csv('result/lgb_result_{0}.csv'.format(20181003),index = False)

p = test_pred_pd[['grid_id','day','hour','car_number']]

## XGB模型

In [134]:
xgb_model = xgb.XGBRegressor(n_estimators=900)

In [135]:
#xgb_model.fit(X_train,train_y)
xgb_model.fit(train_x,train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=900,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [137]:
pre_train = xgb_model.predict(train_x)
print ("modele: LGB RMSE :",np.sqrt(metrics.mean_squared_error(train_y, pre_train)))
print ("modele: LGB RMSE CEIL:",np.sqrt(metrics.mean_squared_error(train_y, np.ceil(pre_train))))
print ("modele: LGB RMSE round:",np.sqrt(metrics.mean_squared_error(train_y, np.round(pre_train))))
print ("modele: LGB RMSE floor:",np.sqrt(metrics.mean_squared_error(train_y, np.floor(pre_train))))

modele: LGB RMSE : 0.7116176903431104
modele: LGB RMSE CEIL: 0.9215226972965197
modele: LGB RMSE round: 0.7609339052527921
modele: LGB RMSE floor: 0.9221757767721435


In [136]:
#pre = xgb_model.predict(X_test)
pre = xgb_model.predict(test_x)

In [138]:
test_pred_pd = test_data[['grid_id','ymd','hour']].copy()
test_pred_pd['car_number'] = 0

In [139]:
test_pred_pd['car_number'] = np.ceil(pre)

In [140]:
test_pred_pd['day'] = pd.to_datetime(test_pred_pd.ymd, format='%Y-%m-%d').dt.strftime('%Y%m%d')

In [141]:
test_pred_pd = test_pred_pd.sort_values(by=['day','hour','grid_id'])

In [142]:
test_pred_pd[['grid_id','day','hour','car_number']].to_csv('result/xgb_result_{0}.csv'.format(20181003),index = False)

In [143]:
p = test_pred_pd[['grid_id','day','hour','car_number']]

### 去除0特征

In [209]:
xgb_fc_importance=analyze_importances(xgb_model.feature_importances_,fc)


In [229]:
fc_zero=xgb_fc_importance[xgb_fc_importance['weight']==0]['feature'].values
print(fc_zero)

fc3=fc.copy()
for ii in fc_zero:
    fc3.remove(ii)

['grid_id_45' 'grid_id_2' 'grid_id_1' 'grid_id_44' 'grid_id_43'
 'grid_id_37' 'grid_id_30' 'grid_id_33' 'grid_id_5' 'grid_id_29'
 'grid_id_26' 'grid_id_23' 'grid_id_22' 'grid_id_21' 'grid_id_4'
 'holiday_1' 'holiday_0' 'hour_13' 'weather_6' 'weather_4'
 'school_holiday_0' 'school_holiday_1' 'school_holiday_bf_af_1' 'hour_15'
 'grid_id_14' 'grid_id_7' 'grid_id_49' 'grid_id_12' 'grid_id_11'
 'grid_id_10' 'grid_id_9' 'grid_id_8' 'grid_id_6']


In [230]:
test_data = all_data2[all_data2.ymd >= '2017-03-13']

train_y2 = train_all_data['y'].values
train_x2 = train_all_data[fc3]
test_x2 = test_data[fc3]

In [212]:
xgb_model2 = xgb.XGBRegressor(n_estimators=750)

#xgb_model.fit(X_train,train_y)
xgb_model2.fit(train_x2,train_y2)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=750,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [213]:
pre_train2 = xgb_model2.predict(train_x2)
print ("modele: LGB RMSE :",np.sqrt(metrics.mean_squared_error(train_y2, pre_train2)))
print ("modele: LGB RMSE CEIL:",np.sqrt(metrics.mean_squared_error(train_y2, np.ceil(pre_train2))))
print ("modele: LGB RMSE round:",np.sqrt(metrics.mean_squared_error(train_y2, np.round(pre_train2))))
print ("modele: LGB RMSE floor:",np.sqrt(metrics.mean_squared_error(train_y2, np.floor(pre_train2))))

modele: LGB RMSE : 0.7249102864721092
modele: LGB RMSE CEIL: 0.9319827448696107
modele: LGB RMSE round: 0.7732254205730746
modele: LGB RMSE floor: 0.9322673698996398


In [232]:
model_predict(xgb_model2,test_x2,"xgb", 20181004)


## 整合xgb-lgb

In [237]:
pd_lgb=pd.read_csv("result/lgb_result_20181004.csv")
pd_xgb=pd.read_csv("result/xgb_result_20181004.csv")

In [242]:
pd_out=pd_lgb.copy()


In [246]:
pd_out['car_number']=np.ceil((pd_xgb['car_number']+pd_lgb['car_number'])/2)
pd_out.to_csv("result/merge_lgb_xgb_mean_20181004_ceil.csv", index=False)


In [247]:
pd_out['car_number']=np.floor((pd_xgb['car_number']+pd_lgb['car_number'])/2)
pd_out.to_csv("result/merge_lgb_xgb_mean_20181004_floor.csv", index=False)
